In [3]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

# Настройки
LOGIN = "your_login_elibrary"
PASSWORD = "your_password_elibrary"
ORG_ID = "id_organization"
MAX_PAGES = 5
OUTPUT_DIR = "elibrary_data"

# Создаем папку для результатов
os.makedirs(OUTPUT_DIR, exist_ok=True)

def initialize_browser():
    """Инициализация нового экземпляра браузера."""
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(options=options)
    return driver

def login_elibrary(driver):
    """Авторизация на eLibrary."""
    print("Начало авторизации...")
    driver.get("https://www.elibrary.ru/")
    
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "login"))
        ).send_keys(LOGIN)
        
        driver.find_element(By.ID, "password").send_keys(PASSWORD)
        driver.find_element(By.XPATH, "//div[@class='butred' and contains(text(), 'Вход')]").click()
        time.sleep(3)
        print("Авторизация успешна")
    except Exception as e:
        print(f"Ошибка авторизации: {e}")
        driver.quit()
        exit()

def solve_captcha(driver):
    """Попытка решить капчу, если она появилась."""
    try:
        WebDriverWait(driver, 5).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//iframe[contains(@title, 'тест Тьюринга')]")))
        checkbox = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='recaptcha-checkbox-border']")))
        checkbox.click()
        print("Капча решена")
        driver.switch_to.default_content()
        return True
    except:
        driver.switch_to.default_content()
        return False

def parse_page(driver):
    """Парсинг статей на текущей странице."""
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    articles = []
    
    table = soup.find('table', {'id': 'restab'})
    if not table:
        return articles
    
    for row in table.find_all('tr')[2:]:  # Пропускаем заголовки
        cols = row.find_all('td')
        if len(cols) < 2:
            continue
        
        title_link = cols[1].find('a', href=True)
        if not title_link or 'id=' not in title_link['href']:
            continue
            
        pub_id = title_link['href'].split('id=')[1].split('&')[0]
        title = title_link.get_text(strip=True)
        authors = cols[1].find('i').get_text(strip=True) if cols[1].find('i') else "Нет авторов"
        
        articles.append({
            'ID': pub_id,
            'Название': title,
            'Авторы': authors
        })
    
    return articles

def parse_with_filter(driver, filter_value, filter_name):
    """Основная функция парсинга с заданным фильтром."""
    print(f"\nНачинаем парсинг с фильтром: {filter_name}")
    
    # Устанавливаем фильтр
    select = Select(WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "show_option"))))
    select.select_by_value(filter_value)
    time.sleep(5)  # Важно дать время на применение фильтра
    
    articles = []
    page_num = 1
    
    while page_num <= MAX_PAGES:
        print(f"Обработка страницы {page_num}...")
        
        # Парсим текущую страницу
        try:
            page_articles = parse_page(driver)
            if not page_articles:
                print("Не найдено статей на странице")
                break
                
            articles.extend(page_articles)
            
            # Сохраняем промежуточные результаты каждые 5 страниц
            if page_num % 5 == 0:
                pd.DataFrame(articles).to_csv(
                    f"{OUTPUT_DIR}/{filter_name}_page_{page_num}.csv",
                    index=False,
                    encoding='utf-8-sig'
                )
                print(f"Промежуточное сохранение: {len(articles)} статей")
                
        except Exception as e:
            print(f"Ошибка парсинга страницы {page_num}: {e}")
            break
        
        # Переход на следующую страницу
        try:
            next_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, 
                "//a[@title='Следующая страница' and contains(@href, 'javascript:goto_page')]")))
            next_btn.click()
            solve_captcha(driver)
            time.sleep(3)
            page_num += 1
        except:
            print("Достигнут конец списка или ошибка перехода")
            break
    
    # Финальное сохранение
    output_file = f"{OUTPUT_DIR}/{filter_name}_final.csv"
    pd.DataFrame(articles).to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"Завершено. Всего {len(articles)} статей сохранено в {output_file}")
    
    return articles

def main():
    all_articles = []
    rinc_articles = []

    try:
        # Этап 1: Парсинг всех публикаций
        print("\nЭтап 1: Парсинг всех публикаций")
        driver = initialize_browser()
        login_elibrary(driver)  # Авторизация
        driver.get(f"https://elibrary.ru/org_items.asp?orgsid={ORG_ID}")
        solve_captcha(driver)
        time.sleep(3)

        all_articles = parse_with_filter(driver, "1", "all_publications")
        driver.quit()  # Закрываем браузер после завершения первого этапа

        # Этап 2: Парсинг публикаций РИНЦ
        print("\nЭтап 2: Парсинг публикаций РИНЦ")
        driver = initialize_browser()
        login_elibrary(driver)  # Авторизация заново
        driver.get(f"https://elibrary.ru/org_items.asp?orgsid={ORG_ID}")
        solve_captcha(driver)
        time.sleep(3)

        rinc_articles = parse_with_filter(driver, "0", "rinc_publications")
        driver.quit()  # Закрываем браузер после завершения второго этапа

    finally:
        # Финальное сохранение результатов
        if all_articles or rinc_articles:
            print("\nСоздание итогового файла...")
            rinc_ids = {art['ID'] for art in rinc_articles}

            final_data = []
            for article in all_articles:
                article['В_РИНЦ'] = 'Да' if article['ID'] in rinc_ids else 'Нет'
                final_data.append(article)

            final_file = f"{OUTPUT_DIR}/final_result.csv"
            pd.DataFrame(final_data).to_csv(final_file, index=False, encoding='utf-8-sig')

            print("\nРезультаты:")
            print(f"Всего публикаций: {len(all_articles)}")
            print(f"Публикаций в РИНЦ: {len(rinc_articles)}")
            print(f"Итоговый файл сохранен как: {final_file}")

if __name__ == "__main__":
    main()


Этап 1: Парсинг всех публикаций
Начало авторизации...
Авторизация успешна

Начинаем парсинг с фильтром: all_publications
Обработка страницы 1...
Обработка страницы 2...
Обработка страницы 3...
Обработка страницы 4...
Обработка страницы 5...
Промежуточное сохранение: 500 статей
Завершено. Всего 500 статей сохранено в elibrary_data/all_publications_final.csv

Этап 2: Парсинг публикаций РИНЦ
Начало авторизации...
Авторизация успешна

Начинаем парсинг с фильтром: rinc_publications
Обработка страницы 1...
Обработка страницы 2...
Обработка страницы 3...
Обработка страницы 4...
Обработка страницы 5...
Промежуточное сохранение: 500 статей
Завершено. Всего 500 статей сохранено в elibrary_data/rinc_publications_final.csv

Создание итогового файла...

Результаты:
Всего публикаций: 500
Публикаций в РИНЦ: 500
Итоговый файл сохранен как: elibrary_data/final_result.csv
